# Preprocessing

In [284]:
%pip install pydriller

In [285]:
from pydriller import Repository
import os
import datetime as dt
from datetime import timedelta, datetime
import numpy as np

In [286]:
# Clone code from the repo and save it for code portability -> via normal git clone
url = "https://github.com/apache/kafka"
repo_path = os.path.join(os.getcwd(), 'kafka')
clone = f"git clone {url} {repo_path}" 

os.system(clone) # Cloning

128

In [287]:
# checkout the tag 3.6.0
os.chdir(repo_path)
os.system("git checkout 3.6.0")
# back to the "home" folder
os.chdir("..")

In [288]:
from_tag = "3.5.1"
from_date=dt.datetime(2023, 7, 14, 18, 51, 0)

to_tag = "3.6.0"
to_date=dt.datetime(2023, 9, 29, 6, 56, 0)

# Using datetimes of the releases, since tags don't yield any commits
# Open question 
# repo = Repository(path_to_repo=repo_path, from_tag=to_tag, to_tag=from_tag)
repo = Repository(path_to_repo=repo_path, since=from_date, to=to_date)

# Exercise 3

In [289]:
# Create a matrix of all current java files
import os

def get_unique_java_files(directory):
    unique_files = set()
    for root, dirs, files in os.walk(directory):
        # print(root)
        # print(files)
        for file in files:
            if file.endswith(".java") or True: #Hotfix
                file_path = os.path.join(root, file)
                # remove the repo path from the file path
                file_path = file_path.replace(directory+"\\", "")
                unique_files.add(file_path)
    return list(unique_files)

# Replace 'your_repo_directory' with the path to your repository
unique_files = get_unique_java_files(repo_path)

In [290]:
def add_file_to_MATRIX(file):
    if file not in FILE_INDEX_MAP:
        print("Adding file to MATRIX: ", file)
        
        new_index = len(FILE_INDEX_MAP)
        FILE_INDEX_MAP[file] = new_index
        unique_files.append(file)  # Update the unique_files list
        
        for window in MATRIX:
            # Correctly expand each matrix to include the new file
            current_matrix = MATRIX[window]
            new_matrix_size = len(unique_files)
            new_matrix = np.zeros((new_matrix_size, new_matrix_size))
            # Copy the old matrix values into the new matrix
            new_matrix[:current_matrix.shape[0], :current_matrix.shape[1]] = current_matrix
            MATRIX[window] = new_matrix


In [291]:
#add them to shared indexes
def process_files(files):
    modified_file_indices = []
    for file in files:
        filename = file.new_path if file.new_path else file.old_path
        # TODO if you want to select only java files, do it there!
        if(not filename.endswith(".java")):
            continue
        # print(file.new_path if file.new_path else file.old_path)
        # Accessing filename correctly 
        add_file_to_MATRIX(filename) #check if file is already in the MATRIX, if not add it (issue for files which were created and are not in repo now)
        modified_file_indices.append(FILE_INDEX_MAP[filename])
    return modified_file_indices

In [292]:
def update_MATRIX2(files_current, files_others, window):
    if(len(files_current) == 0):
        return
    if(len(files_others) == 0):
        return
    
    assert all(not isinstance(item, list) for item in files_current)
    assert all(not isinstance(item, list) for item in files_others)

    for i in files_current:
            for j in files_others:
                if i != j:  # Skip incrementing for the same file
                    # if((i == 5773) or j == 5773):
                    #     print("Updating MATRIX for window: ", window, " for files: ", i, j)
                    #     print("pevious value: ", MATRIX[window][i][j])
                    #     print("next value", MATRIX[window][i][j] + 1)
                    MATRIX[window][i][j] += 1
                         

In [294]:
time_windows = [0, 24, 48, 72, 168]  # time windows in hours
MATRIX = {window: np.zeros((len(unique_files), len(unique_files))) for window in time_windows}
FILE_INDEX_MAP = {file: index for index, file in enumerate(unique_files)}


In [295]:
# Debugging
# SECONDFILE = FILE_INDEX_MAP["core\\src\\test\\java\\kafka\\log\\remote\\RemoteLogManagerTest.java"]
# print(SECONDFILE)
# MAINFILE = FILE_INDEX_MAP["core\\src\\main\\java\\kafka\\log\\remote\\RemoteLogManager.java"]
# print(MAINFILE)


1439

In [298]:
def process_time_windows(commit_queue, current_commit, time_windows):
    current_commit_time, files, hash = current_commit
    
    # run for each window 
    for window in time_windows:
            
        adjacent_files = []

        for commit_date, file_list, commit_hash in commit_queue:
            if timedelta(hours=0) <= commit_date - current_commit_time <= timedelta(hours=window):
                adjacent_files.extend(file_list)

        update_MATRIX2(files, adjacent_files, window)

In [299]:
from collections import deque
from datetime import timedelta
all_commits = Repository(path_to_repo=repo_path, 
                        #  only_modifications_with_file_types=['.java'],
                        #  since=datetime(2022, 9, 1, 00, 59, 0)
                         ).traverse_commits()

commit_queue = deque()
for commit in all_commits:
        commit_data = (commit.committer_date, process_files(commit.modified_files), commit.hash)
        commit_queue.append(commit_data)

        # Check if the time span exceeds 168 hours
        while (commit_queue[-1][0] - commit_queue[0][0] > timedelta(hours=168)):

            current_commit = commit_queue[0]
            process_time_windows(commit_queue, current_commit, time_windows)
            commit_queue.popleft()
        # otherwise just fill it in the loop

# Process the remaining commits
while len(commit_queue) > 0:
    current_commit = commit_queue[0]
    process_time_windows(commit_queue, current_commit, time_windows)
    commit_queue.popleft()
                

Adding file to MATRIX:  contrib\hadoop-consumer\src\main\java\kafka\etl\KafkaETLContext.java
Adding file to MATRIX:  contrib\hadoop-consumer\src\main\java\kafka\etl\KafkaETLInputFormat.java
Adding file to MATRIX:  contrib\hadoop-consumer\src\main\java\kafka\etl\KafkaETLJob.java
Adding file to MATRIX:  contrib\hadoop-consumer\src\main\java\kafka\etl\KafkaETLKey.java
Adding file to MATRIX:  contrib\hadoop-consumer\src\main\java\kafka\etl\KafkaETLRecordReader.java
Adding file to MATRIX:  contrib\hadoop-consumer\src\main\java\kafka\etl\KafkaETLRequest.java
Adding file to MATRIX:  contrib\hadoop-consumer\src\main\java\kafka\etl\KafkaETLUtils.java
Adding file to MATRIX:  contrib\hadoop-consumer\src\main\java\kafka\etl\Props.java
Adding file to MATRIX:  contrib\hadoop-consumer\src\main\java\kafka\etl\UndefinedPropertyException.java
Adding file to MATRIX:  contrib\hadoop-consumer\src\main\java\kafka\etl\impl\DataGenerator.java
Adding file to MATRIX:  contrib\hadoop-consumer\src\main\java\kafka

In [300]:
import pandas as pd

def MATRIX_to_dataframe(MATRIX, unique_files):
    df_list = []
    for window, matrix in MATRIX.items():
        df = pd.DataFrame(matrix, index=unique_files, columns=unique_files)
        df = df.stack().reset_index()
        df.columns = ['File1', 'File2', 'Count']
        df['Time_Window'] = window
        df_list.append(df)

    return pd.concat(df_list, ignore_index=True)

result_df = MATRIX_to_dataframe(MATRIX, unique_files)

In [ ]:
# Filter pairs with more than 5 changes
result_df = result_df[result_df['Count'] > 5]

# Sort, reset index, etc.
result_df = result_df.sort_values(by=['Count'], ascending=False).reset_index(drop=True)

In [311]:
# make file 1 and file2 display the whole name
pd.set_option('display.max_colwidth', None)
result_df.head(30)


,File1,File2,Count,Time_Window
0,clients\src\main\java\org\apache\kafka\common\record\MemoryRecords.java,streams\src\test\java\org\apache\kafka\streams\integration\RegexSourceIntegrationTest.java,6.0,72
1,clients\src\test\java\org\apache\kafka\common\utils\UtilsTest.java,streams\src\main\java\org\apache\kafka\streams\kstream\UnlimitedWindows.java,6.0,168
2,streams\src\test\java\org\apache\kafka\streams\kstream\internals\KStreamKStreamJoinTest.java,clients\src\main\java\org\apache\kafka\common\requests\ControlledShutdownRequest.java,6.0,48
3,streams\src\test\java\org\apache\kafka\streams\TopologyTest.java,streams\src\test\java\org\apache\kafka\streams\state\internals\ChangeLoggingSessionBytesStoreTest.java,6.0,168
4,streams\src\main\java\org\apache\kafka\streams\state\Stores.java,streams\src\main\java\org\apache\kafka\streams\kstream\ValueMapper.java,6.0,168
5,streams\src\main\java\org\apache\kafka\streams\state\internals\MeteredSessionStore.java,streams\src\main\java\org\apache\kafka\streams\state\internals\MemoryNavigableLRUCache.java,6.0,72
6,streams\test-utils\src\main\java\org\apache\kafka\streams\TopologyTestDriver.java,streams\src\test\java\org\apache\kafka\streams\integration\StoreQueryIntegrationTest.java,6.0,24
7,streams\src\test\java\org\apache\kafka\test\NoOpProcessorContext.java,streams\src\main\java\org\apache\kafka\streams\state\internals\ChangeLoggingWindowBytesStore.java,6.0,0
8,clients\src\main\java\org\apache\kafka\common\requests\SyncGroupResponse.java,clients\src\main\java\org\apache\kafka\common\record\Record.java,6.0,168
9,streams\src\main\java\org\apache\kafka\streams\StreamsBuilder.java,streams\src\test\java\org\apache\kafka\streams\processor\internals\InternalTopicManagerTest.java,6.0,48


In [316]:
import json
def create_json_from_dataframe(dataframe, json_file_path):
    # This function will create a JSON structure based on the dataframe provided
    # Pre-filter rows where 'Time_Window' is not 0
    filtered_df = dataframe[dataframe['Time_Window'] != 0]

    # Function to process each group
    def process_group(group):
        file_pair = [group.name[0], group.name[1]]
        coupled_commits = group[['Time_Window', 'Count']].to_dict('records')
        return {"file_pair": file_pair, "coupled_commits": coupled_commits}

    # Group by 'File1' and 'File2' and apply processing function
    json_data = filtered_df.groupby(['File1', 'File2']).apply(process_group).tolist()

    with open(json_file_path, 'w') as json_file:
        json.dump(json_data, json_file, indent=2)

    print(f"JSON data has been written to {json_file_path}")

# create_json_from_dataframe(result_df, "ex_3.json") #Uncomment to write inside

JSON data has been written to ex_3.json


In [313]:
# export pandas into csv
# result_df.to_csv("ex_3_smaller.csv") #uncomment to write inside

In [319]:
# create the data json with joint commits
def create_json_joint_from_dataframe(dataframe, json_file_path):
    # Filter rows where 'Time_Window' is 0
    filtered_df = dataframe[dataframe['Time_Window'] == 0]

    # Function to calculate logical coupling
    def calculate_logical_coupling(group):
        joint_commits = group['Count'].sum()
        commits_with_others_file1 = filtered_df[filtered_df['File1'] == group.name[0]]['Count'].sum() - joint_commits
        commits_with_others_file2 = filtered_df[filtered_df['File2'] == group.name[1]]['Count'].sum() - joint_commits
        return {
            "Joint": joint_commits,
            group.name[0]: commits_with_others_file1,
            group.name[1]: commits_with_others_file2
        }

    # Function to process each group for the new JSON structure
    def process_group(group):
        file_pair = [group.name[0], group.name[1]]
        logical_coupling = calculate_logical_coupling(group)
        return {"file_pair": file_pair, "logical_coupling": logical_coupling}

    # Group by 'File1' and 'File2' and apply processing function
    json_data = filtered_df.groupby(['File1', 'File2']).apply(process_group).tolist()

    with open(json_file_path, 'w') as json_file:
        json.dump(json_data, json_file, indent=2)

    print(f"JSON data has been written to {json_file_path}")

create_json_joint_from_dataframe(result_df, "ex_3_2.json") #Uncomment to write inside

JSON data has been written to ex_3_2.json
